In [6]:
import numpy as np
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [3]:
import PIL
import tensorflow as tf
import random
import re
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.ops import enable_eager_execution
#disable_eager_execution()
enable_eager_execution()

In [4]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
import tensorflow_datasets as tfds

In [8]:
from __future__ import print_function
import argparse
from tensorflow.keras.layers import Input
import scipy.misc

import tensorflow_model_optimization as tfmot

In [9]:
class DefaultBNQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):

    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass
    def set_quantize_activations(self, layer, quantize_activations):
        pass
    def get_output_quantizers(self, layer):
        return [tfmot.quantization.keras.quantizers.MovingAverageQuantizer(num_bits=8, per_axis=False, symmetric=False, narrow_range=False)]

    def get_config(self):
        return {}
    
    
class NoOpQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    """Use this config object if the layer has nothing to be quantized for 
    quantization aware training."""

    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass

    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        # Does not quantize output, since we return an empty list.
        return []

    def get_config(self):
        return {}
    
    
def apply_quantization(layer):
    if 'bn'  in layer.name:
        return tfmot.quantization.keras.quantize_annotate_layer(layer,DefaultBNQuantizeConfig())
    elif 'concat' in layer.name:
        return tfmot.quantization.keras.quantize_annotate_layer(layer,NoOpQuantizeConfig())
    else:
        return tfmot.quantization.keras.quantize_annotate_layer(layer)

In [10]:
def preprocess_image(features):
    """Preprocesses the given image.
       will convert the images from RGB to BGR, then will zero-center each color channel with respect to the ImageNet dataset, without scaling.
       mean = [103.939, 116.779, 123.68]
       std = None
  """
    image = features["image"]
    image = tf.image.resize(image,[224,224])
    features["image"] = image
    return features

# Load Data

In [11]:
BATCH_SIZE = 50
tfds_dataset2, tfds_info  = tfds.load(name='imagenet2012_subset', split='validation[-60%:]', with_info=True,
                                     data_dir='../datasets/ImageNet/') # use the last 20% of images among 50000 validation images for testing
#tf.compat.v1.data.make_one_shot_iterator(tfds_dataset1).get_next()

In [12]:
val_ds = tfds_dataset2.map(preprocess_image).batch(BATCH_SIZE)

In [13]:
len(val_ds)

600

# Loading models

In [15]:
# input image dimensions
img_rows, img_cols = 224 ,224
input_shape = (img_rows, img_cols, 3)

In [19]:
res_model_ = ResNet50(input_shape=input_shape)
res_model = ResNet50(input_tensor = res_model_.input)
res_p_model = ResNet50(input_tensor=res_model_.input)
res_pqat_model = tfmot.quantization.keras.quantize_model(res_model_)

In [21]:
res_model.load_weights("../weights/fp_model_40_resnet50.h5")
res_p_model.load_weights("../weights/p_model_40_resnet50.h5")
res_pqat_model.load_weights("../weights/pqat_model_40_resnet50.h5")
res_model.trainable = False
res_p_model.trainable = False
res_pqat_model.trainable = False
res_model.compile()
res_p_model.compile()
res_pqat_model.compile()
print("Res Done")

Res Done


In [25]:
mob_model_ = MobileNet(input_shape=input_shape)
mob_model = MobileNet(input_tensor = mob_model_.input)
mob_p_model = MobileNet(input_tensor=mob_model_.input)
mob_pqat_model = tfmot.quantization.keras.quantize_model(mob_model_)

In [26]:
mob_model.load_weights("../weights/fp_model_40_mobilenet.h5")
mob_p_model.load_weights("../weights/p_model_40_mobilenet.h5")
mob_pqat_model.load_weights("../weights/pqat_model_40_mobilenet.h5")
mob_model.trainable = False
mob_p_model.trainable = False
mob_pqat_model.trainable = False
mob_model.compile()
mob_p_model.compile()
mob_pqat_model.compile()
print("Mob Done")

Mob Done


In [29]:
den_model_ = DenseNet121(input_shape=input_shape)
den_model = DenseNet121(input_tensor = den_model_.input)
den_p_model = DenseNet121(input_tensor=den_model_.input)

In [30]:
# Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` 
# to the layers of the model.
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer
annotated_model = tf.keras.models.clone_model(
    den_model_,
    clone_function=apply_quantization,
)

with tfmot.quantization.keras.quantize_scope({'DefaultBNQuantizeConfig': DefaultBNQuantizeConfig, 'NoOpQuantizeConfig': NoOpQuantizeConfig}):
    den_pqat_model = tfmot.quantization.keras.quantize_apply(annotated_model)

den_model.load_weights("../weights/fp_model_40_densenet121.h5")
den_p_model.load_weights("../weights/p_model_40_densenet121.h5")
den_pqat_model.load_weights("../weights/pqat_model_40_densenet121.h5")
den_model.trainable = False
den_p_model.trainable = False
den_pqat_model.trainable = False
den_model.compile()
den_p_model.compile()
den_pqat_model.compile()
print("Den Done")

Den Done


In [31]:
label_map = {}

In [32]:
def check(label_map, number):
    progress = 0
    for i in label_map.keys():
        progress += len(label_map[i])
    
    print(progress/(number*1000))

In [36]:
# find images that all models agree on
def work(image,file,label):
    
    # res
    
    res_orig_logist = res_model.predict(tf.keras.applications.resnet.preprocess_input(image.copy())[None, ...])
    res_p_logist = res_p_model.predict(tf.keras.applications.resnet.preprocess_input(image.copy())[None, ...])
    res_pqat_logist = res_pqat_model.predict(tf.keras.applications.resnet.preprocess_input(image.copy())[None, ...])
    label1 = np.argmax(res_orig_logist)
    label2 = np.argmax(res_p_logist)
    label3 = np.argmax(res_pqat_logist)
    
    if not (label1 == label2 == label3 == label):
        return False
    
    # mobile

    image_copy = np.copy(image)

    mob_orig_logist = mob_model.predict(tf.keras.applications.mobilenet.preprocess_input(image.copy())[None, ...])
    mob_p_logist = mob_p_model.predict(tf.keras.applications.mobilenet.preprocess_input(image.copy())[None, ...])
    mob_pqat_logist = mob_pqat_model.predict(tf.keras.applications.mobilenet.preprocess_input(image.copy())[None, ...])
    label4 = np.argmax(mob_orig_logist)
    label5 = np.argmax(mob_p_logist)
    label6 = np.argmax(mob_pqat_logist)
    
    if not (label4 == label5 == label6 == label):
        return False
    
    # dense
    
    image_copy = np.copy(image)
    
    den_orig_logist = den_model.predict(tf.keras.applications.densenet.preprocess_input(image.copy())[None, ...])
    den_p_logist = den_p_model.predict(tf.keras.applications.densenet.preprocess_input(image.copy())[None, ...])
    den_pqat_logists = den_pqat_model.predict(tf.keras.applications.densenet.preprocess_input(image.copy())[None, ...])
    label7 = np.argmax(den_orig_logist)
    label8 = np.argmax(den_p_logist)
    label9 = np.argmax(den_pqat_logists)
    
    if not (label7 == label8 == label9 == label):
        return False
    
    return True

In [ ]:
num_per_class = 3 # number of images per class

for i,images in enumerate(val_ds):
    
    print("% OF IMAGES SEEN: "+str(i/600))
    
    for j, image in enumerate(images['image']):

        image = images['image'][j].numpy()
        file = images['file_name'][j].numpy()
        label = images['label'][j].numpy()
        
        if label not in label_map:
            label_map[label] = []
            
        if work(image,file,label):
            label_map[label] += [(image,file,label)]

In [46]:
# k: v -> v classes has k images that all models agree on
d = {}
for c in label_map:
    d[len(label_map[c])] = d.get(len(label_map[c]), 0) + 1
print(dict(sorted(d.items(), key=lambda x:x[0])))

{0: 45, 1: 42, 2: 62, 3: 69, 4: 69, 5: 48, 6: 76, 7: 66, 8: 81, 9: 59, 10: 55, 11: 42, 12: 58, 13: 41, 14: 30, 15: 38, 16: 24, 17: 19, 18: 20, 19: 9, 20: 12, 21: 14, 22: 4, 23: 9, 24: 5, 25: 1, 27: 1, 30: 1}


In [ ]:
new_label_map = {}
extra = []
for c in label_map:
    if len(label_map[c]) >= num_per_class:
        new_label_map[c] = label_map[c][:num_per_class]
        if len(label_map[c]) > num_per_class:
            extra.append((c, label_map[c][num_per_class]))
    else:
        new_label_map[c] = label_map[c]
        for i in range(num_per_class-len(label_map[c])):
            print(i, len(extra))
            new_label_map[extra[0][0]].append(extra[0][1])
            extra = extra[1:]

In [51]:
# k: v -> v classes has k images that all models agree on
d = {}
for c in new_label_map:
    d[len(new_label_map[c])] = d.get(len(new_label_map[c]), 0) + 1
print(dict(sorted(d.items(), key=lambda x:x[0])))

{0: 45, 1: 42, 2: 62, 3: 570, 4: 281}


In [52]:
cnt = 0
for c in new_label_map:
    cnt += len(new_label_map[c])
print(cnt)

3000


In [54]:
file_data = []
image_data = []
label_data = []

In [55]:
for i in new_label_map:
    try:
        a = [s[0] for s in new_label_map[i]]
        b = [s[1] for s in new_label_map[i]]
        c = [s[2] for s in new_label_map[i]]
        image_data = image_data + a
        file_data = file_data + b
        label_data = label_data + c
    except:
        print(i)


In [124]:
print(len(image_data))

file_data_ = np.array(file_data)
image_data_ = np.array(image_data)
label_data_ = np.array(label_data)

3000


In [125]:
KImagePerClass = tf.data.Dataset.from_tensor_slices({"file_name":file_data_,"image":image_data_, "label":label_data_})

In [126]:
tf.data.experimental.save(
    KImagePerClass, "../datasets/ImageNet/pruning/3kImages", compression=None, shard_func=None
)

In [127]:
print(KImagePerClass.element_spec)

{'file_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}
